In [ ]:
import libsequence
import sys
import pandas as pd
import math
import argparse
import vcf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Function to create dictionary of polySIM summary statistics. Returns dictionary of summary stats
def get_polySIM_stats(sd):
    #Create polysim object
    ps = libsequence.PolySIM(sd)
    #Create list of methods (ie polySIM summaryStats)
    a = [method for method in dir(ps) if callable(getattr(ps, method)) if not method.startswith('_')]
    #Loop through methods, storing names as keys, and estimates as values in dict
    ss_dict = {}
    for method in a:
        ss_dict[method] = getattr(ps, method)()
        
    return(ss_dict)

In [ ]:
#Function to create dictionary of LD stats. Returns dictionary.
def get_LD_stats(sd):
    ld = libsequence.ld(sd)
    df = pd.DataFrame(ld)
    ss_dict = {}
    try:
        ss_dict['meanrsq'] = sum(df['rsq'])/len(df['rsq'])
        ss_dict['meanD'] = sum(df['D'])/len(df['D'])
        ss_dict['meanDprime'] = sum(df['Dprime'])/len(df['Dprime'])
    except Exception:
        ss_dict['meanrsq'] = np.nan
        ss_dict['meanD'] = np.nan
        ss_dict['meanDprime'] = np.nan
    return(ss_dict)

In [ ]:
#Define chromosome lengths
d_lens = {
    'NC_071442.1':216975769,
    'NC_071443.1':202808461,
    'NC_071444.1':189455076,
    'NC_071445.1':173414057,
    'NC_071446.1':161716902,
    'NC_071447.1':159674559,
    'NC_071448.1':156129252,
    'NC_071449.1':126104592,
    'NC_071450.1':132900640,
    'NC_071451.1':136971485,
    'NC_071452.1':128529401,
    'NC_071453.1':123697827,
    'NC_071454.1':117638787,
    'NC_071455.1':112969529,
    'NC_071456.1':98621277,
    'NC_071457.1':98110366,
    'NC_071458.1':74700100,
    'NC_071459.1':47063576,
    'NC_071460.1':50540917,
    'NC_071461.1':44412365,
    'NC_071462.1':50614742,
    'NC_071463.1':49900457
}

In [ ]:
#Create bed files for 10kb windows. Bedtools intersect will be used on these to calculate number of unmasked sites.
for chrom in d_lens.keys():
    lst = []
    for i in range(0, d_lens[chrom]-5000, 5000):
        lst.append([chrom, i, i+10000])
    df = pd.DataFrame(lst, columns=['#chr','start','end'])
    #Ensure final window ends at chromosome end coordinate
    df['end'] = np.where(df.end <=d_lens[chrom], df.end, d_lens[chrom])
    df.to_csv("/home/vivak/chimerism/marmoset/empirical/vcf/demog/" + chrom + "_10kb.bed", header=True, sep='\t',
                                                            index=False)

In [ ]:
#Calculate genome-wide stats
#Create empty list to store input for pylibseq (this will be a nested list)
l_data = []
for chrom in d_lens.keys():
    #subset mask df for chromosome and bin coordinates by windows
    mdf = pd.read_csv("/home/vivak/chimerism/marmoset/empirical/vcf/demog/masked_"+chrom+"_10kb.bed", names=['#chr','start','end','accessible_sites'], sep='\t')
    #Load vcf into python
    vcf_reader = vcf.Reader(filename=r'/home/vivak/chimerism/marmoset/empirical/vcf/demog/ALL_'+chrom+'.snps.GT.GATK_BP.DP.repeat_constrained_exons_10kb_mask.vcf.gz')
    

    #Loop through records in vcf
    for record in vcf_reader:
        #Create list of relative position (index 0), and genotype as string (index 1)
        l_data.append([record.POS, ''.join([x['GT'] for x in record.samples]).replace('/','').replace('|','')])

#Create libseq object from l_data
sd = libsequence.SimData(l_data)

d = {}
d = {**d, **get_polySIM_stats(sd)}
for stat in ['nhaps', 'numexternalmutations', 'numpoly', 'numsingletons','rm', 'thetah', 'thetal', 'thetapi', 'thetaw']:
                    d[stat] = d[stat] / 213936105
df = pd.DataFrame.from_dict(d, orient='index').T
df

In [ ]:
#Calculate stats across 10kb sliding windows
sdf = pd.DataFrame()
stepSize = 5000
winSize = 10000
for chrom in d_lens.keys():
    #subset mask df for chromosome and bin coordinates by windows
    mdf = pd.read_csv("/home/vivak/chimerism/marmoset/empirical/vcf/demog/masked_"+chrom+"_10kb.bed", names=['#chr','start','end','accessible_sites'], sep='\t')
    #Load vcf into python
    vcf_reader = vcf.Reader(filename=r'/home/vivak/chimerism/marmoset/empirical/vcf/demog/ALL_'+chrom+'.snps.GT.GATK_BP.DP.repeat_constrained_exons_10kb_mask.vcf.gz')
    
    #Create empty list to store input for pylibseq (this will be a nested list)
    l_data = []
    positions = []
    #Loop through records in vcf
    for record in vcf_reader:
        #Create list of relative position (index 0), and genotype as string (index 1)
        l_data.append([record.POS, ''.join([x['GT'] for x in record.samples]).replace('/','').replace('|','')])
        #positions.append([record.POS/d_lens[chrom],record.POS])

    #Create libseq object from l_data
    sd = libsequence.SimData(l_data)
    #define sliding windows
    wins = libsequence.Windows(sd,window_size=winSize,step_len=stepSize,starting_pos=0,ending_pos=d_lens[chrom])
    #Create a variable for number of windows
    num_wins = len(wins)
    
    #Create empty df which will be our final output format
    df = pd.DataFrame()
    
    #Loop through windows
    for k, win in enumerate(wins):
        if((win.size()>5) & (mdf.iloc[k].accessible_sites>50)):
            #Create empty dict to store results
            wd = {}
            #Add window number to dict
            wd['chrom'] = chrom
            wd['window'] = k
            S = len([x for x in l_data if (x[0]>=mdf.iloc[k].start) & (x[0]<=mdf.iloc[k].end)])/mdf.iloc[k].accessible_sites
            if(S>0):
                wd['S'] = S
                wd = {**wd, **get_polySIM_stats(win)}
                for stat in ['nhaps', 'numexternalmutations', 'numpoly', 'numsingletons','rm', 'thetah', 'thetal', 'thetapi', 'thetaw']:
                    wd[stat] = wd[stat] / mdf.iloc[k].accessible_sites
                #For LD things are slightly more complex as libseq throws out an error if there aren't enough SNPs. We can avoid this with an if statement
                # if len(win.pos()) >= 5: #LD stats are pairwise. If only 1 site exists, it'll show an error.
                #     #Add all LD stats to dict
                #     wd = {**wd, **get_LD_stats(win)}
                # else:
                #     #Otherwise use 'NA' or you can use np.nan
                #     wd['meanrsq'] = np.nan
                #     wd['meanD'] = np.nan
                #     wd['meanDprime'] = np.nan
                #Convert dict into temp df        
                tdf = pd.DataFrame.from_dict(wd, orient='index').T
                #Concatenate temp df with main output df
                df = pd.concat([df, tdf])
    sdf = pd.concat([sdf, df])
    #Write stats df out to file
    df.to_csv("/home/vivak/chimerism/marmoset/empirical/stats/demog/" + chrom + "_10kb.stats", header=True, sep='\t', index=False)
sdf.to_csv("/home/vivak/chimerism/marmoset/empirical/stats/demog/concat_10kb.stats", header=True, sep='\t', index=False)

In [ ]:
#Get means and stds of each stat for ABC analysis
sdf = pd.read_csv("/home/vivak/chimerism/marmoset/empirical/stats/demog/concat_10kb.stats", header=0, sep='\t')
sdf = sdf[sdf.numpoly>0]
means = []
stds = []
for stat in sdf.columns[2:]:
    means.append(sdf[stat].mean())
    stds.append(sdf[stat].std())
    
df = pd.DataFrame(means+stds).T
df.columns = [x+'_m' for x in sdf.columns[2:]] + [x+'_std' for x in sdf.columns[2:]]
df.to_csv("/home/vivak/chimerism/marmoset/empirical/stats/demog/10kb_ABC.stats", header=True, sep='\t', index=False)